## Loading

In [179]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [180]:
df_orig = pd.read_spss("./Data/private_landlord_survey/UKDA-8455-spss/spss/spss24/epls_2018_forarchive.sav")

In [181]:
df = df_orig
df = df[["serial_2", # Unique Identifier
        "LLCat1", # LL org category
        "LLeth2cat", # LL ethnicity
        "Age4cat", # LL Age
        "ValperPrpA", # Avg mkt val per prop
        "MktValgrpA", # Mkt val all props
        "BTL", # Buy to let loan/other
        "LnperPrpA", # Avg loan per prop
        "LnValgrpA", # Total val all loans
        "LTVgrpA", # LTV market val
        "TmLLgrpB", # How long been LL
        "LLExp", # Length experience as LL
        "NumPrpgrp", # How many props
        "RtIncgrpa", # Gross rental inc
        "LLIncgrpA", # Total LL inc no rent
        "TotIncgrp", # Total LL inc
        "RtPrIncgrpA", # Rent as prop. inc
        "Port_size", # Portfolio size
        "LLEmpl01", # Emp. Status: FT
        "LLEmpl02", # Emp. Status: PT
        "LLEmpl03", # Emp. Status: Self-employed as LL
        "LLEmpl04", # Emp. Status: Self-employed
        "LLEmpl05", # Emp. Status: Retired
        "LLEmpl06", # Emp. Status: Company director
        "LLEmpl07", # Emp. Status: Student
        "LLEmpl08", # Emp. Status: FT carer
        "LLEmpl09", # Emp. Status: Unemployed
        "LLOrg1", # LL Org: National LLs Assc.
        "LLOrg2", # LL Org: Residential LL Assc.
        "LLOrg3", # LL Org: Other professional
        "LLOrg4", # LL Org: None
        "AgOrg1", # Rental Org: ARLA
        "AgOrg2", # Rental Org: NAEA
        "AgOrg3", # Rental Org: NALS
        "AgOrg4", # Rental Org: UKALA
        "AgOrg5", # Rental Org: Other
        "AgOrg6", # Rental Org: None
        "Regprop01", # Prop Region: East England
        "Regprop02", # Prop Region: East Midlands
        "Regprop03", # Prop Region: Inner London
        "Regprop04", # Prop Region: Outer London
        "Regprop05", # Prop Region: North East
        "Regprop06", # Prop Region: North West
        "Regprop07", # Prop Region: South East
        "Regprop08", # Prop Region: South West
        "Regprop09", # Prop Region: West Midlands
        "Regprop10", # Prop Region: Yorkshire/Humber
        "LLInfo01", # Where info: Letting Agent
        "LLInfo02", # Where info: LL membership ass. or org
        "LLInfo03", # Where info: Online landlord forums/websites
        "LLInfo04", # Where info: Gov't websites
        "LLInfo05", # Where info: Family/friends
        "LLInfo06", # Where info: Online media
        "LLInfo07", # Where info: TV
        "LLInfo08", # Where info: Radio
        "LLInfo09", # Where info: Newspapers
        "AgtuseA1", # Agent use: For letting svcs
        "AgtuseA2", # Agent use: For mgmt svcs
        "AgtuseA3", # Agent use: None
        "FutVac", # Next Vacancy will relet
        "FutProp", # Next two years, plan
        "FutRLv1", # LL leave/reduce: Financial
        "FutRLv2", # LL leave/reduce: Personal
        "FutRLv3", # LL leave/reduce: Legislative
        "FutRLv4", # LL leave/reduce: Other
        "FutRLv5", # LL leave/reduce: None
        "FutRLv6", # LL leave/reduce: Don't Know
        "FutInc1", # LL Increase/maintain: Financial
        "FutInc2", # LL Increase/maintain: Personal
        "FutInc3", # LL Increase/maintain: Legislative
        "FutInc4", # LL Increase/maintain: Other
        "FutInc5", # LL Increase/maintain: None
        "FutInc6", # LL Increase/maintain: Don't know
        "LLPmb1", # LL Serious problems: Financial
        "LLPmb2", # LL Serious problems: Legislative
        "LLPmb3", # LL Serious problems: Tenant behaviour
        "LLPmb4", # LL Serious problems: Other
        "LLPmb5", # LL Serious problems: None
        "Lntyp1", # Current loan: B2L mort
        "Lntyp2", # Current loan: Commercial loan
        "Lntyp3", # Current loan: Family/friends
        "Lntyp4", # Current loan: No debt
        "Lntyp5", # Current loan: Other
        "TaxB201", # Awareness: Stamp duty increase
        "TaxB202", # Awareness: Reduced tax relief
        "TaxB203", # Awareness: Reduced CG tax non-prop
        "TaxB204", # Awareness: Change wear/tear allow
        "TaxB205", # Awareness: Change minimum rent B2L mort
        "TaxB206", # Awareness: Letting agent fee ban
        "TaxB207", # Awareness: Tax treatment foreign prop own
        "TaxB208", # Awareness: None
        "TaxB301", # As above but understanding
        "TaxB302",
        "TaxB303",
        "TaxB304",
        "TaxB305",
        "TaxB306",
        "TaxB307",
        "EPCEFG", # Any props. w/ EPC rating E, F or G
        "EPCE", # Awareness: EPC requirements
        "LetReq03"] # Requirement: EPC Certificate]
]

df["serial_2"] = df["serial_2"].astype("int")

In [182]:
def get_stats(df: pd.DataFrame, col: str, percent:bool = True) -> pd.DataFrame:
    df_stats = df[col].value_counts().to_frame()
    if percent == True:
        f = lambda a : round(a / len(df[df[col].notna()].index), 4)
        df_stats = df_stats.applymap(f)
    return df_stats

In [183]:
len(df[df['FutRLv3'] == 'Yes'])/len(df[df['FutRLv3'].notna()])

0.6599552572706935

In [184]:
get_stats(df, 'FutRLv3')

,count
FutRLv3,
Yes,0.66
No,0.34


## Imputing Variables

In [185]:
def calculate_stlt(prop_val, num_prps):
    if prop_val <= 250_000:
        sdlt = prop_val * 0.03
    elif prop_val <= 925_000:
        sdlt = 250_000 * 0.03 + \
            (prop_val - 250_000) * 0.08
    elif prop_val <= 1_500_000:
        sdlt = 250_000 * 0.03 + \
            675_000 * 0.08 + \
            (prop_val - 925_000) * 0.13
    else:
        sdlt = 250_000 * 0.03 + \
            675_000 * 0.08 + \
            925_000 * 0.13 + \
            (prop_val - 1_500_000) * 0.15
    
    sdlt_total = sdlt * num_prps
    
    return sdlt_total

In [186]:
def est_cap_gains(years_owned, num_prps, mkt_val):
    cg = lambda x : (-0.0006 * x ** 3 + 0.0314 * x ** 2 - 0.5628 * x + 3.7054) + 1
    
    cg_amts = []
    years_per_prp = years_owned / num_prps
    current_yr = years_per_prp
    
    cg_amts.append(cg(24 - years_owned))
    
    if num_prps > 1:
        for i in range(num_prps - 1):
            cg_amts.append(cg(current_yr))
            current_yr += years_per_prp
    
    cg_avg = sum(cg_amts) / len(cg_amts)
    
    orig_mkt = mkt_val / (cg_avg)
    cg_total = round(max(0, mkt_val - orig_mkt) * 0.28, 2)
    
    
    return cg_total

In [187]:
df["LTVgrpA"] = df.LTVgrpA.cat.add_categories(['0%'])
df.loc[df["Lntyp4"] == "Yes", "LTVgrpA"] = "0%"
get_stats(df, "LTVgrpA")

,count
LTVgrpA,
0%,0.3645
50 to 59%,0.1098
60 to 69%,0.0947
40 to 49%,0.0875
30 to 39%,0.0719
20 to 29%,0.0627
70 to 79%,0.0578
1 to 9%,0.0493
10 to 19%,0.0481


In [188]:
band_mapping = {
    '0%': 0,
    '1 to 9%': 0.05,
    '10 to 19%': 0.15,
    '20 to 29%': 0.25,
    '30 to 39%': 0.35,
    '40 to 49%': 0.45,
    '50 to 59%': 0.55,
    '60 to 69%': 0.65,
    '70 to 79%': 0.75,
    '80 to 89%': 0.85,
    '90 to 99%': 0.95,
    '100 and above': 1.0
}

df['LTVgrpA_estimated'] = df['LTVgrpA'].map(band_mapping)
df['LTVgrpA_estimated'] = pd.to_numeric(df['LTVgrpA_estimated'], errors='coerce')

In [204]:
band_mapping = {
    'less than £100,000': 50000,
    '£100,000 to £199,999': 150000,
    '£200,000 to £499,999': 350000,
    '£500,000 to £999,999': 750000,
    '£1,000,000 to £1,999,999': 1500000,
    '£2,000,000 to £2,999,999': 2500000,
    '£3,000,000 to £3,999,999': 3500000,
    '£4,000,000 to £4,999,999': 4500000,
    '£5,000,000 to £5,999,999': 5500000,
    '£6,000,000 or more': 7000000
}
df['MktValgrpA_estimated'] = df['MktValgrpA'].map(band_mapping)
df['MktValgrpA_estimated'] = pd.to_numeric(df['MktValgrpA_estimated'], errors='coerce')

In [205]:
band_mapping = {
    '1 only': 1,
    '2 to 4': 2,
    '5 to 9': 5,
    '10 to 24': 15,
    '25 to 100': 62,
    'more than 100': 150
}

df['NumPrpgrp_estimated'] = df['NumPrpgrp'].map(band_mapping)
df['NumPrpgrp_estimated'] = pd.to_numeric(df['NumPrpgrp_estimated'], errors='coerce')

In [206]:
band_mapping = {
    'up to £49,999': 25000,
    '£50,000 to £79,999': 65000,
    '£80,000 to £99,999': 90000,
    '£100,000 to £129,999': 115000,
    '£130,000 to £149,999': 140000,
    '£150,000 to £179,999': 165000,
    '£180,000 to £199,999': 190000,
    '£200,000 to £499,999': 350000,
    '£500,000 and over': 600000
}
df['ValperPrpA_estimated'] = df['ValperPrpA'].map(band_mapping)
df['ValperPrpA_estimated'] = pd.to_numeric(df['ValperPrpA_estimated'], errors='coerce')

In [207]:
df = df[df['ValperPrpA_estimated'].notna()]

In [244]:
def define_max(numprpgrp):
    prop_max = {
        '1 only': 1,
        '2 to 4': 4,
        '5 to 9': 9,
        '10 to 24': 24,
        '25 to 100': 100,
        'more than 100': 1000
    }
    return prop_max.get(numprpgrp)

df['max_props'] = df.apply(lambda row: define_max(row['NumPrpgrp']), axis=1)
    

In [249]:
df['Num_prp_estimated'] = (df['MktValgrpA_estimated'] / df['ValperPrpA_estimated']).astype(int)
df.loc[df['NumPrpgrp'] == '1 only', 'Num_prp_estimated'] = 1
df['Num_prp_estimated'] = df.apply(lambda row: row['max_props'] if row['Num_prp_estimated'] > row['max_props'] else row['Num_prp_estimated'], axis=1)
df['Num_prp_estimated'] = pd.to_numeric(df['Num_prp_estimated'], errors='coerce')

In [250]:
df['mkt_val_est_result'] = df['ValperPrpA_estimated'] * df['Num_prp_estimated']
df['mkt_val_est_result'] = pd.to_numeric(df['mkt_val_est_result'], errors='coerce')

In [251]:
df['Debt_estimated'] = df['LTVgrpA_estimated'] * df['mkt_val_est_result']
df['Debt_estimated'] = pd.to_numeric(df['Debt_estimated'], errors='coerce')

In [252]:
df['sdlt_estimated'] = df.apply(lambda row: calculate_stlt(row['Num_prp_estimated'], row['ValperPrpA_estimated']), axis=1)
df['sdlt_estimated'] = pd.to_numeric(df['sdlt_estimated'], errors='coerce')

In [253]:
years_owned = {
    'Less than 2 years': 1,  # 1 year, assuming an average of 1 year
    '3 to 5 years': 3,  # 3 years, skewed lower from 4 years
    '6 to 10 years': 7,  # 7 years, skewed lower from 8 years
    '11 to 20 years': 13,  # 13 years, skewed lower from 15 years
    'over 20 years': 20,  # 22 years, skewed lower from 25 years
    }

df['YearsOwned_estimated'] = df['TmLLgrpB'].map(years_owned)
df['YearsOwned_estimated'] = pd.to_numeric(df['YearsOwned_estimated'], errors='coerce')
df['PrpsPerYear'] = df['Num_prp_estimated'] / df['YearsOwned_estimated']
df['PrpsPerYear'] = pd.to_numeric(df['PrpsPerYear'], errors='coerce')

In [269]:
df['cg_estimated'] = df.apply(lambda row: est_cap_gains(row['YearsOwned_estimated'], row['Num_prp_estimated'], row['mkt_val_est_result']), axis=1)
df['cg_estimated'] = pd.to_numeric(df['cg_estimated'], errors='coerce')
df['total_lending'] = df['Debt_estimated'].fillna(0) + df['sdlt_estimated'].fillna(0) + df['cg_estimated'].fillna(0)
df['new_ltv'] = df['total_lending'] / df['mkt_val_est_result']

In [275]:
df['Num_prp_estimated'].sum()

31518

## New Data

In [277]:
df_test = df[[
    'LLCat1',
    'TmLLgrpB',
    'LTVgrpA_estimated',
    'Debt_estimated',
    'Num_prp_estimated',
    'NumPrpgrp_estimated',
    'ValperPrpA_estimated',
    'mkt_val_est_result',
    'sdlt_estimated',
    'cg_estimated',
    'total_lending',
    'new_ltv',
]]

df_test['Debt_estimated'] = df_test['Debt_estimated'].fillna(0)

In [278]:
tgt_mkt = df_test[(df_test["LLCat1"].isin(['As an individual or group of individuals']))]
# tgt_mkt = tgt_mkt[tgt_mkt['new_ltv'] <= .8]
tgt_len = len(tgt_mkt.index)
df_len = len(df_test[(df_test["LLCat1"].notna())].index)

print(f" Target market: {tgt_len} \n Total market: {df_len} \n Percentage of total individuals: {round((tgt_len/df_len)*100,2)}%")

 Target market: 5286 
 Total market: 5787 
 Percentage of total individuals: 91.34%


In [279]:
tgt_mkt['total_lending'].mean()

495824.7692678774

In [280]:
tgt_mkt_prps = tgt_mkt['Num_prp_estimated'].sum()
mkt_prps = df['Num_prp_estimated'].sum()

print(f" Target market: {tgt_mkt_prps} \n Total market: {mkt_prps} \n Percentage of total properties: {round((tgt_mkt_prps/mkt_prps)*100,2)}%")

 Target market: 24656 
 Total market: 31518 
 Percentage of total properties: 78.23%


In [281]:
df['Num_prp_estimated'].mean()

5.446345256609642

In [282]:
tgt_mkt_val = tgt_mkt['mkt_val_est_result'].sum()
mkt_val = df_test['mkt_val_est_result'].sum()

print(f" Target market: {tgt_mkt_val} \n Total market: {mkt_val} \n Percentage of total market value: {round((tgt_mkt_val/mkt_val)*100,2)}%")

 Target market: 5252105000 
 Total market: 6430440000 
 Percentage of total market value: 81.68%


## Further specifying Tgt Mkt

In [285]:
tgt_mkt = df_test[(df_test["LLCat1"].isin(['As an individual or group of individuals']))]
tgt_mkt = tgt_mkt[tgt_mkt["Num_prp_estimated"] <= 10]
# tgt_mkt = tgt_mkt[tgt_mkt["Num_prp_estimated"] >= 5]
tgt_mkt = tgt_mkt[tgt_mkt['new_ltv'] <= .65]
tgt_mkt = tgt_mkt[tgt_mkt['new_ltv'] >= .60]
tgt_mkt['total_lending'].sum()

177279107.98

In [286]:
tgt_mkt.sample(5)

,LLCat1,TmLLgrpB,LTVgrpA_estimated,Debt_estimated,Num_prp_estimated,NumPrpgrp_estimated,ValperPrpA_estimated,mkt_val_est_result,sdlt_estimated,cg_estimated,total_lending,new_ltv
4384,As an individual or group of individuals,11 to 20 years,0.45,1417500.0,9,5,350000,3150000,94500.0,502451.15,2014451.15,0.639508
6870,As an individual or group of individuals,over 20 years,0.45,598500.0,7,5,190000,1330000,39900.0,193076.86,831476.86,0.625171
5656,As an individual or group of individuals,11 to 20 years,0.45,1080000.0,4,2,600000,2400000,72000.0,357936.82,1509936.82,0.629140
5858,As an individual or group of individuals,6 to 10 years,0.45,155250.0,3,2,115000,345000,10350.0,58072.90,223672.90,0.648327
1235,As an individual or group of individuals,6 to 10 years,0.55,192500.0,1,1,350000,350000,10500.0,20505.14,223505.14,0.638586


In [287]:
round(tgt_mkt['total_lending'].mean(), 2)

590930.36

In [289]:
tgt_mkt_val = tgt_mkt['mkt_val_est_result'].sum()
mkt_val = df_test['mkt_val_est_result'].sum()

print(f" Target market: {tgt_mkt_val} \n Total market: {mkt_val} \n Percentage of total market value: {round((tgt_mkt_val/mkt_val)*100,2)}%")

 Target market: 281395000 
 Total market: 6430440000 
 Percentage of total market value: 4.38%


In [290]:
tgt_mkt_prps = tgt_mkt['Num_prp_estimated'].sum()
mkt_prps = df['Num_prp_estimated'].sum()

print(f" Target market: {tgt_mkt_prps} \n Total market: {mkt_prps} \n Percentage of total properties: {round((tgt_mkt_prps/mkt_prps)*100,2)}%")

 Target market: 1015 
 Total market: 31518 
 Percentage of total properties: 3.22%


In [291]:
tgt_len = len(tgt_mkt.index)
df_len = len(df_test[(df_test["LLCat1"].notna())].index)

print(f" Target market: {tgt_len} \n Total market: {df_len} \n Percentage of total individuals: {round((tgt_len/df_len)*100,2)}%")

 Target market: 300 
 Total market: 5787 
 Percentage of total individuals: 5.18%


## New Stats

In [294]:
df_test = df[[
    'LLCat1',
    'Age4cat',
    'LLEmpl03',
    'TmLLgrpB',
    'LTVgrpA_estimated',
    'Debt_estimated',
    'Num_prp_estimated',
    'ValperPrpA_estimated',
    'mkt_val_est_result',
    'sdlt_estimated',
    'cg_estimated',
    'total_lending',
    'new_ltv',
]]

df_test['Debt_estimated'] = df_test['Debt_estimated'].fillna(0)

Total Unincorporated LLs

In [295]:
df_tam = df_test[(df_test["LLCat1"].isin(['As an individual or group of individuals', 'Other (please specify)']))]

In [296]:
df_tam["Num_prp_estimated"].mean()

4.859607267388512

In [306]:
(df_tam['Num_prp_estimated'].sum()/df_test['Num_prp_estimated'].sum()) * 4900000

4116758.677581065

In [299]:
mkt_pc = len(df_tam) / len(df)
mkt_pc

0.9415932261966476

Self-employed LLs

In [307]:
df_sam = df_tam[(df_tam['LLEmpl03']=="Yes")]

In [308]:
df_sam["Num_prp_estimated"].mean()

8.862555066079295

In [311]:
(df_sam['Num_prp_estimated'].sum()/df_test['Num_prp_estimated'].sum()) * 4900000

1563839.710641538

Self-employed under 45

In [54]:
df_som = df_sam[(df_sam["Age4cat"].isin(['35-44', '18-34']))]

In [55]:
df_som["Num_prp_estimated"].mean()

7.350877192982456

In [56]:
mkt_pc = len(df_som) / len(df)
mkt_pc * 1_500_000

29548.989113530326